In [1]:
# python imports
from glob import glob
import ctypes
import struct
import sys
from pathlib import Path
# standard scientific packages
import numpy as np
import pandas
# repository specific modules
import vtu
PyVtu = vtu.PyVtu
import small_functions
import ts_auto_wrapper
TSWrapper = ts_auto_wrapper.TSWrapper

In [2]:
# import cffi # :( haven't managed to make it work

Test the wrapper: since the location of the trisurf library is not known, the module defines `TSWrapper` class which acts like a module. Both the module and the class have `help` documentation.  
*The path to trisurf need to be changed*

In [3]:
path_to_trisurf = Path("/opt/workspace/msc_project/cluster-trisurf")
ts=TSWrapper(path_to_trisurf)
help(ts)

Help on TSWrapper in module ts_auto_wrapper object:

class TSWrapper(builtins.object)
 |  TSWrapper(path_to_trisurf='/opt/workspace/msc_project/cluster-trisurf', more_types=None)
 |  
 |  Class that instantiate a trisurf wrapper from a path.
 |  
 |  Design to act like a module i.e.
 |  >>> from ts_auto_wrapper import TSWrapper
 |  >>> ts = TSWrapper('/path/to/trisurf/project/trisurf_ng')
 |  Exposes python binding to the library using CDLL.
 |  Everything is available in ts.X, but they are also organized by types with faster autocompletes:
 |      ts.ts_types: classes for types (ts_vertex, ts_tape, ts_vesicle, ...)
 |      ts.functions: functions (init_vertex, parseDump, ...)
 |      ts.globals: global variables
 |      ts.enums: enum definitions
 |      ts.TS_...: several #defines are hardcoded (e.g. TS_SUCCESS)
 |  and misc. things: ctype function POINTER, pointer; a pretty_print, and a byte_to_int function
 |  
 |  The raw functions with 'restype' and 'argtypes' are available as in

We can compare an example vtu file between a `PyVtu` object from `vtu.py` versus the wrappers `ctypes` results.  
*The .vtu file need to be changed. The file should match the trisurf version*

In [5]:
example_vtu_file = '/mnt/c/Users/yoavr/Desktop/timestep_000087.vtu'
v = PyVtu(example_vtu_file)
vesicle = ts.parseDump(example_vtu_file) # pointer to the generated vesicle
vesicle

The wrapper for trisurf `parseDump` return a `ctype` class: this represent a pointer (`LP_to_`) to a struct (`ts_vesicle`).

We can test Samo's original version instead.  
*The path to trisurf nede to be changed*

In [6]:
# path_to_trisurf = '/opt/workspace/msc_project/trisurf_samo/trisurf-ng'
# ts = TSWrapper(path_to_trisurf)
# example_vtu_file = '/opt/workspace/msc_project/simulations/QA_tests/cluster_version/feature_tests/test_wrapper_with_old_version/timestep_000000.vtu'
# vesicle = ts.parseDump(example_vtu_file)
# v = PyVtu(example_vtu_file)

Trisurf heavily uses *pointers-to-structs* `vesicle->vlist->vtx[6]->x`. The ctype translation is with .contents: `vesicle.contents.vlist.contents[6].contents.x`.

In [7]:
print("type of vesicle center mass ",vesicle.contents.cm) # vesicle { ts_double cm[3] ...}
vesicle.contents.cm[:]

type of vesicle center mass  <ts_auto_wrapper.c_double_Array_3 object at 0x7f2f9b62a7c0>


[0.0, 0.0, 0.0]

We can use other functions in the library like `parsetape`.  
The CDLL functions have set `argtypes` and `restype`, and are wrapped with a few more processing:
* for strings, like paths to files, functions that need character arrays can recive strings.  
* for out parameters, functions that need pointer to doubles instantiate and return them.  

The docstring of the function list the actual, pre-proccessing signature and the post-processing return type. The "Bare" c function is available in `ts._c_`*function*

In [8]:
ts.parsetape?

Signature:
ts.parsetape(
    *args,
    _base_c_function=<_FuncPtr object at 0x7f2f9ddb7040>,
    arg_types=[<class 'ctypes.LP_c_char'>],
    process_args=[<function TSWrapper.__init__.<locals>.process_str_to_char_p at 0x7f300bdcd630>],
    process_out=[<function TSWrapper.__init__.<locals>.process_neutral at 0x7f300bdccee0>, None],
)
Docstring: Wrapper for parsetape with arguments , filename: str or __str__. Return <class 'ts_auto_wrapper.LP_ts_tape'>
File:      /opt/workspace/msc_project/lab_note_and_py/trisurf-python/ts_auto_wrapper.py
Type:      function

We can use trisurf to parse the default tape in the `/src/` folder  
This works because `ts.parsetape` knows to first convert the input object to string with `str()` (in this case, a `Path` )

In [9]:
base_tape=ts.parsetape(ts.path_to_trisurf/'src/tape')
base_tape

We can look at the tape using this helper function: this is meant to more easily print the tape or other ctypes structures

In [10]:
print(ts.pretty_text(base_tape))

    tape_text <ctypes.LP_c_char object at 0x7f2f9b62a8c0>
    R_nucleus 0.0
    R_nucleusX 0.0
    R_nucleusY 0.0
    R_nucleusZ 0.0
    xkA0 1.0
    xkV0 1.0
    V0 0.0
    A0 0.0
    Vfraction 1.0
    constvolprecision 1e-14
    xk0 20.0
    xk2 -20.0
    dmax 1.7
    dmin_interspecies 1.2
    stepsize 0.15
    kspring 800.0
    xi 100.0
    pressure 0.0
    c0 0.5
    d0 0.5
    w 1.0
    F 1.0
    plane_d 10.0
    plane_F 1.0
    vicsek_strength 0.1
    vicsek_radius 4.0
    adhesion_z -5.0
    adhesion_cutoff 1.0
    adhesion_strength 1.0
    adhesion_radius 5.0
    adhesion_scale 5.0
    adhesion_factor 2.0
    min_dihedral_angle_cosine 0.1
    mcsweeps 1000
    random_seed 0
    iterations 10
    inititer 0
    nshell 10
    ncxmax 100
    ncymax 100
    nczmax 100
    number_of_vertices_with_c0 50
    npoly 0
    nmono 20
    internal_poly 0
    nfil 0
    nfono 3
    shc 0
    pressure_switch b'\x00'
    volume_switch b'\x00'
    area_switch b'\x00'
    quiet b'\x00'
    plane

Let's compare the wrapper tape with the `small_functions.py` tape options extraction.

In [11]:
tstape = ts.pretty_text(base_tape)
with open(ts.path_to_trisurf/'src/tape','r') as f:
    pytape = small_functions.get_tape_options(f.read())
all_lines = {}
for line1 in tstape.splitlines():
    first=line1.split()[0]
    if first in pytape:
        all_lines[first] = [line1, pytape[first]]
    else:
        all_lines[first] = [line1, None]
pandas.DataFrame(all_lines, index=['c','python'])

,<ts_auto_wrapper.LP_ts_tape,tape_text,R_nucleus,R_nucleusX,R_nucleusY,R_nucleusZ,xkA0,xkV0,V0,A0,...,area_switch,quiet,plane_confinement_switch,allow_center_mass_movement,force_balance_along_z_axis,adhesion_geometry,adhesion_model,bond_model,curvature_model,force_model
c,<ts_auto_wrapper.LP_ts_tape object at 0x7f2f9b...,tape_text <ctypes.LP_c_char object at 0x7f...,R_nucleus 0.0,R_nucleusX 0.0,R_nucleusY 0.0,R_nucleusZ 0.0,xkA0 1.0,xkV0 1.0,V0 0.0,A0 0.0,...,area_switch b'\x00',quiet b'\x00',plane_confinement_switch b'\x00',allow_center_mass_movement b'\x00',force_balance_along_z_axis b'\x00',adhesion_geometry b'\x00',adhesion_model b'\x00',bond_model b'\x00',curvature_model b'\x0f',force_model b'\x00'
python,None,None,0,0,0,0,1.0,1.0,0,0,...,0,false,0,0,0,0,0,0,15,0


Both agree on their values of the options (this is not trivial, since structure misalignment can corrupt the values)

Here we look at the spontaneous curvature of each vertex `vtx.c` with id between 50 to 100. This is not exactly ergonomic.

In [12]:
np.array([vesicle.contents.vlist.contents.vtx[i].contents.c for i in range(50,100)])

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

We can compare the mean curvature from trisurf versus the mean curvature from the .vtu. There may be factors of 2.

In [13]:
ts.functions.vesicle_meancurvature(vesicle), v.mean_curvature.sum()

(193.32805182985842, 386.65610365971753)

In [15]:
ts.functions._c_direct_force_energy.argtypes

[ts_auto_wrapper.LP_ts_vesicle,
 ts_auto_wrapper.LP_ts_vertex,
 ts_auto_wrapper.LP_ts_vertex]

Here we look at the force acting on a vertex directly based on the CDLL:  
* we take the first active vertex `vtx`
* we create three dummy vertices with position at 1 unit before
* we calculate the work on each vertex with using the dummy vertex as the starting position
* $W=-f\cdot dx$, so we get the force in each direction
* we can compare it with the PyVtu

In [16]:
first_active = v.indices[v.type==47][0]

vtx=vesicle.contents.vlist.contents.vtx[first_active]
print(f"z in first active vertex:\nvlist->vtx[{first_active}]->z={vtx.contents.z}",f"\nPyVtu z:",v.pos[first_active,2])

z in first active vertex:
vlist->vtx[4]->z=10.551420071625163 
PyVtu z: 10.551420071625163


In [17]:
# dummy starting positions: dx=x̂, dx=ŷ dx=ẑ
vtx_oldx=ts.ts_vertex(x=vtx.contents.x-1, y=vtx.contents.y,   z=vtx.contents.z)
vtx_oldy=ts.ts_vertex(x=vtx.contents.x,   y=vtx.contents.y-1, z=vtx.contents.z)
vtx_oldz=ts.ts_vertex(x=vtx.contents.x,   y=vtx.contents.y,   z=vtx.contents.z-1)

In [18]:
fx = -ts.direct_force_energy(vesicle,vtx,ctypes.pointer(vtx_oldx)) # using the wrapper function

fy = -ts.functions._c_direct_force_energy(vesicle,vtx,ctypes.pointer(vtx_oldy)) # using the underlying cdell function with restype and argtype

ts.cdll.direct_force_energy.restype=ctypes.c_double # using the cdll function directly
fz = -ts.cdll.direct_force_energy(vesicle, vtx, ctypes.pointer(vtx_oldz))

print("trisurf force:",fx,fy,fz,"vtu force: ",v.force[first_active])

trisurf force: 0.23226609732205358 0.3289169843523071 0.2964221271082282 vtu force:  [0.22445575 0.33483795 0.29580933]


Check we are the same position as in the PyVtu:

In [19]:
ts_auto_wrapper.parse_type("int **func",ts.ts_types)

('int', {'func': ts_auto_wrapper.LP_LP_c_int})

In [20]:
ts.byte_to_int(vesicle.contents.tape.contents.bond_model)

0

In [21]:
ts.dec_funcs['next_idx']

('ts_idx', ctypes.c_int, 'ts_idx i, ts_idx max')

In [22]:
ts.functions._c_next_idx.restype

ctypes.c_int

In [23]:
ts.functions.prev_small(4,8)

3

In [24]:
ts.dec_funcs['parsetape']

('ts_tape', ts_auto_wrapper.LP_ts_tape, 'char *filename')

In [25]:
cluster = ts.functions.init_cluster_list()

In [26]:
import inspect

In [27]:
def get_default_args(func):
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }

In [28]:
df=get_default_args(ts.functions.clusterize_vesicle)
process_args =df['process_args']
func = df['_base_c_function']

In [29]:
func(vesicle,cluster)

b'\x00'

In [30]:
[f(x) for f,x in zip(process_args,(vesicle,cluster))]

In [34]:
ts.functions._c_clusterize_vesicle.argtypes

[ts_auto_wrapper.LP_ts_vesicle, ts_auto_wrapper.LP_ts_cluster_list]

In [35]:
ts.functions.clusterize_vesicle(vesicle,cluster)

True

In [36]:
cluster.contents.cluster.contents.contents.vtx.contents.contents.c

1.0

In [37]:
ts.functions.clusterize_vesicle

<function ts_auto_wrapper.TSWrapper.__init__.<locals>.f(*args, _base_c_function=<_FuncPtr object at 0x7f2f9ddb5840>, arg_types=[<class 'ts_auto_wrapper.LP_ts_vesicle'>, <class 'ts_auto_wrapper.LP_ts_cluster_list'>], process_args=[<function TSWrapper.__init__.<locals>.process_neutral at 0x7f300bdccee0>, <function TSWrapper.__init__.<locals>.process_neutral at 0x7f300bdccee0>], process_out=[<function TSWrapper.__init__.<locals>.process_ts_bool_to_true_false at 0x7f300bdcd2d0>, None, None])>

In [38]:
ts.functions.single_timestep(vesicle)

(True, 0.5868244323632875, 0.01311160857051487)

In [39]:
ts.functions.vesicle_meancurvature?

Signature:
ts.functions.vesicle_meancurvature(
    *args,
    _base_c_function=<_FuncPtr object at 0x7f2f9dd72b00>,
    arg_types=[<class 'ts_auto_wrapper.LP_ts_vesicle'>],
    process_args=[<function TSWrapper.__init__.<locals>.process_neutral at 0x7f300bdccee0>],
    process_out=[<function TSWrapper.__init__.<locals>.process_neutral at 0x7f300bdccee0>, None],
)
Docstring: Wrapper for vesicle_meancurvature with arguments , vesicle: <class 'ts_auto_wrapper.LP_ts_vesicle'>. Return <class 'ctypes.c_double'>
File:      /opt/workspace/msc_project/lab_note_and_py/trisurf-python/ts_auto_wrapper.py
Type:      function

In [40]:
ts.functions.vesicle_meancurvature(vesicle)

192.85626016870233